In [1]:
import sqlite3
import json
import os
from pathlib import Path
import pandas as pd
from collections import defaultdict

DATA_DIR = Path('data') / 'clean-data'

In [2]:
# Load cartoons from json files
def load_cartoons():
    c.execute('''CREATE TABLE Cartoons
                 (id INT PRIMARY KEY, title TEXT, keywords TEXT, caption TEXT, image_url TEXT, author TEXT, date TEXT)''')

    for filename in os.listdir(DATA_DIR):
        cid, ext = os.path.splitext(filename)

        if ext != '.json':
            continue

        with open(DATA_DIR / filename, 'r') as f:
            metadata = json.load(f)
            values = (int(cid), metadata["title"], metadata["keywords"], metadata["caption"],
                      metadata["image_url"], metadata["author"], metadata["date"])
            c.execute("INSERT INTO Cartoons VALUES (?, ?, ?, ?, ?, ?, ?)", values)

In [3]:
# Load bounding boxes from VGG data
def entity_from_json(s: str):
    d  = json.loads(s)
    if len(d) > 0:
        d = {k: v for (k, v) in d.items() if v != ""}
        assert len(d) == 1
        return next(iter(d.keys()))
    return None

class MissingDict(dict):
    def __missing__(self, key):
        return None

def load_df(path):
    df = pd.read_csv(path)
    
    # id
    df["id"] = df["#filename"].apply(lambda s: int(os.path.splitext(s)[0]))
    
    # bounding box
    df["x"] = df["region_shape_attributes"].apply(lambda s: MissingDict(json.loads(s))["x"])
    df["y"] = df["region_shape_attributes"].apply(lambda s: MissingDict(json.loads(s))["y"])
    df["width"] = df["region_shape_attributes"].apply(lambda s: MissingDict(json.loads(s))["width"])
    df["height"] = df["region_shape_attributes"].apply(lambda s: MissingDict(json.loads(s))["height"])

    # entity
    df["entity"] = df["region_attributes"].apply(entity_from_json)
    
    df = df.filter(["id", "entity", "x", "y", "width", "height"])
    return df

In [4]:
# Load dataframe from files
bounding_box_files = ["data/bounding-boxes/via_region_data.csv", 
                      "data/bounding-boxes/via_region_data2.csv", 
                      "data/bounding-boxes/via_region_data3.csv"]
df = pd.concat([load_df(x) for x in bounding_box_files])
# bounding_box_tuples = list(df.itertuples(index=False, name=None))
# bounding_box_tuples

In [5]:
conn = sqlite3.connect("cartoons.sqlite")

df.to_sql("BoundingBoxes", con=conn, if_exists='replace', index=False)

In [6]:
conn.execute("SELECT * FROM BoundingBoxes").fetchall()

[(704, None, None, None, None, None),
 (2177, None, None, None, None, None),
 (4652, None, None, None, None, None),
 (4830, None, None, None, None, None),
 (10056, None, None, None, None, None),
 (11876, None, None, None, None, None),
 (13630, None, None, None, None, None),
 (14364, None, None, None, None, None),
 (14963, None, None, None, None, None),
 (16119, None, None, None, None, None),
 (16843, None, None, None, None, None),
 (18110, None, None, None, None, None),
 (19461, None, None, None, None, None),
 (19958, None, None, None, None, None),
 (20232, None, None, None, None, None),
 (21673, None, None, None, None, None),
 (23235, None, None, None, None, None),
 (24048, None, None, None, None, None),
 (24657, None, None, None, None, None),
 (36193, None, None, None, None, None),
 (38807, None, None, None, None, None),
 (39913, None, None, None, None, None),
 (42576, None, None, None, None, None),
 (43169, None, None, None, None, None),
 (46583, None, None, None, None, None),
 (473

In [81]:
# c = conn.cursor()

# TODO: add code

# # Save changes and exit
# conn.commit()
# conn.close()
conn.commit()
conn.close()